<a href="https://colab.research.google.com/github/DarioDell/Integration_with_third_party_services/blob/main/Integrar_ChatBot_con_Gmail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Integración del ChatBot con servicios externos

<div style="background-color:#D9EEFF;color:blue;padding:2%;">
<h2>Introducción</h2>

Este es un notebook más que interesante porque explora una de las capacidades más interesantes de los GPTs que es su integración con servicios de terceros. Los LLMs donde muestran su verdadero potencial es cuando le damos acceso a otras fuentes de datos (por ejemplo dentro de una organización) y permitimos que procesen esos datos ayudando a obtener diferentes insigths y patrones dentro de la información.<br>
Se llevará a cabo la integración de un <u>ChatBot(ChatGPT)</u> con el servicio de correo electrónico de <u>Gmail.</u>

La aplicación debe ser capaz de realizar diferentes funciones sobre los correos electrónicos del usuario. Por ejemplo, debe ser capaz de buscar todos los correos electrónicos de un remitente concreto y resumir su contenido. En otras palabras, utilizar al LLM como si fuese un asistente a la hora de leer y procesar los correos electrónicos. Por ejemplo, poder pedirle al LLM "Quiero obtener todos los mails de una determinada persona", "Quiero que obtengas cierta información precisa de un determinado correro", "Muestrame esos datos en el correo de tal forma u otra". Tiene un gran potencial aplicable a distintos casos reales.

</div>

# Resolución del caso práctico

## 0. Instalación de librerías externas

In [ ]:
# La librería necesaria para invocar el LLM

!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4


## 1. Lectura de la API Key

In [ ]:
import openai

with open("/content/drive/MyDrive/api-keys/API_openAI.txt") as f:     # Es necesario crear una clave API en OpenAI
  openai.api_key = f.readline()

## 2. Selección del modelo



A continuación se exploran los distintos roles que proporciona OpenAI para sus LLMs y la función que tienen cada uno de ellos:


*  Rol `user`: Representa al usuario final que interactúa con el LLM a través de un chat.
*  Rol `assistant`: Representa al LLM que estemos utilizando, en este caso, `gpt-3.5-turbo`.
*  Rol `system`: Este rol representa al desarrollador de sistema. Permite proporcionar instrucciones "root" al LLM para que se sigan durante la conversación con el usuario.

In [ ]:
def obtener_completion(mensajes, model="gpt-3.5-turbo"):
  respuesta = openai.ChatCompletion.create(
      model=model,
      messages=mensajes,
      temperature=0, # Este hiperparámetro controla la aleatoriedad del modelo
  )
  return respuesta.choices[0].message["content"]

## 3. Integración con Gmail

In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 87.6 MB/s eta 0:00:00
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.155.0
    Uninstalling google-api-python-client-2.155.0:
      Successfully uninstalled google-api-python-client-2.155.0


In [ ]:
from __future__ import print_function

import os.path
import base64
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']


def obtener_correos(remitente):
    """Proporciona los correos electrónicos del usuario."""
    creds = None
    if os.path.exists('/content/drive/MyDrive/api-keys/token.json'):
        creds = Credentials.from_authorized_user_file('/content/drive/MyDrive/api-keys/token.json', SCOPES)

    try:
        correos = """"""
        service = build('gmail', 'v1', credentials=creds)                                   # Se crea el cliente
        results = service.users().messages().list(userId='me', labelIds=['INBOX']).execute()   # La concuslta de todos los correos electrónicos, me quedo con todos los mails del INBOX independientemente si están abiertos o no
        # Recorremos todos los correos electrónicos
        for msg in results['messages']:
          mensaje = service.users().messages().get(userId='me', id=msg['id']).execute()   # Se extrae el mensaje concreto
          # Comparamos el remitente del correo
          if remitente in str(mensaje['payload']['headers'][14]):
            # Extraemos el contenido del correo
            datos = base64.b64decode(mensaje['payload']['parts'][0]['body']['data'], '-_')
            correos += f"""
'''
Contenido: {datos}
'''
"""
        return correos

    except HttpError as error:
        # TODO(developer) - Handle errors from gmail API.
        print(f'An error occurred: {error}')

In [ ]:
def collect_messages(_):
    prompt = inp.value_input    # Recibe la entrada, lo que introduzca el usuario
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})    # Se toma esa entrada del usuario y se añade al contexto
    response = obtener_completion(context)              # Se le manda el contexto al LLM para que haga la predicción
    try:                                    # Primera respuesta del LLM
      remitente = eval(response)["remitente"]        # eval interpreta al JSON como un diccionario de Python y se queda únicamente con el 'remitente' (el cual tiene la dirección del correo electrónico)
      accion_solicitada = eval(response)["accion_solicitada"]
      correos = obtener_correos(remitente)
      context.pop(0) # Elimino del contexto la regla inicial
      context.append({'role': 'user', 'content': f"Realiza la siguiente acción: {accion_solicitada} sobre los siguientes correos electrónicos: {correos}"})
      response = obtener_completion(context)
    except:
      pass
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))
    return pn.Column(*panels)

In [ ]:
def end_chat(event):
    panels.append(pn.pane.Alert("Chat terminado por el usuario.", alert_type='success'))
    context.append({'role': 'system', 'content':"Despídete del usuario de manera amable y amigable."})
    response = obtener_completion(context)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(    # Se muestra en el panel o interfaz gráfica
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))
    return pn.Column(*panels)


In [ ]:
import panel as pn  # Interfaz gráfica
pn.extension()

panels = []

#Este es el prompt inicial, es lo que va a solicitar el LLM inicialmente, es lo que le paso por contexto

context = [ {'role':'system', 'content':
"""
Debes solicitar al usuario que te indique una dirección de correo electrónico que no sea la suya y la tarea que quiere realizar sobre sus correos electrónicos. \
Con esta información, debes generar un JSON con las siguientes claves. No debes generar nada más.
"remitente":<correo electrónico indicado por el usuario>
"accion_solicitada":<descripción de la accion solicitada por el usuario sobre los correos electrónicos del remitente>
"""} ]


inp = pn.widgets.TextInput(value="Hola", placeholder='Introduce texto aqui...')
button_conversation = pn.widgets.Button(name="Chat!")
button_end_chat = pn.widgets.Button(name="Terminar Chat")

button_end_chat.on_click(end_chat)

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation, button_end_chat),
    pn.panel(interactive_conversation, loading_indicator=True, sizing_mode="stretch_both"),
)

dashboard

Column
    [0] TextInput(placeholder='Introduce texto aqui...')
    [1] Row
        [0] Button(name='Chat!')
        [1] Button(name='Terminar Chat')
    [2] ParamFunction(function, _pane=Column, defer_load=False, loading_indicator=True, sizing_mode='stretch_both')

En el panel de arriba se podrá interactuar con el LLM solicitandole distintas funciones a realizar con los correos electrónicos. Simplemente se introduce en el input una <u>dirección de email y la tarea a realizar</u>. Desde leer emails, hasta que realice un resumen de los mismos con las especificaciones que le pasemos al LLM. También se le puede preguntar al LLM si alguno de los correos es malicioso, que extraíga los nombres propios de un determinado email, que haga clasificaciones, etc. Tiene infinidad de utilidades que se podrían aplicar en un ambiente laboral.